---


# **Neural coding**



---

####*Learning outcomes*

1. Understanding what a **tuning curve** is and how it relates sensory stimuli to neural activity.
2. Knowing how to use a **Poisson process** to model variability in spiking.
3. Understanding the relationship between the **mean, variance, bias and mean squared error** of an estimator.
4. Knowing how to use the **Cramer-Rao** bound to bound the variance of an estimator.

# Introduction

The main function of the brain is to make use of perceptual input to generate relevant behavioral output. In order to do this, it needs to create and manipulate informative representations of the world. Neural coding considers the problem of how information about the world is represented in the neural activity. More specifically, let $s$ be an external stimulus and $\mathbf{r}=(r_1,r_2,...,r_M)$ the neural response it generates (e.g. firing rate of each neuron in a population of size $M$): *encoding* is the process by which the stimulus generates, in a stochastic manner, patterns of neural activity $p(\mathbf{r}|s)$, while *decoding* is the reverse process, by which a neural population is “read out”, either by an experimenter or by downstream neurons, to produce an estimate of the stimulus $p(s|\mathbf{r})$.

For a single neuron, different values of $r$ will be recorded when $s$ is presented many times, and the mean response, denoted by $\left<r\right>=f(s)$, is called the *tuning curve* of the neuron. This function can typically be monotonic or bell‐shaped (e.g. for stimuli like orientation or position in space). When it is bell‐shaped, then the peak of the function is called the *preferred stimulus* of the neuron.

Generally, different neurons have different tuning curves. Consider a set of neurons sensitive to a feature of the stimulus (e.g. neurons with different preferred orientations), then the information about the stimulus can be represented by the collective activity of this set of neurons. This is called a *population code*, and it can be useful for increasing the animal's certainty about a feature, as well as for encoding multiple features at once.

In this lab we will focus on the decoding problem and compare different strategies of estimating, given a pattern of activity $\mathbf{r}$, the stimulus $s$ that generated the response. We will therefore assume an encoding model $p(\mathbf{r}|s)$ and ask how well we can estimate $\hat{s}$ from the observed population response.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import scipy
sns.set()

# global defaults for plots - optional
sns.set_theme(style="ticks",
              palette="Set2",
              font_scale=1.7,
              rc={
              "axes.spines.right": False,
              "axes.spines.top": False,
          },
          )

# Exercise 1.1 - Population of Poisson Neurons

We first simulate one trial of the response of a population of (independent) Poisson neurons to stimulus *s*, which we will then try to decode. We consider the case of a circular stimulus $s\in\left[-180^\circ, 180^\circ\right)$ (e.g. a visual bar oriented at a certain angle or the head direction of an animal). We model each neuron $i$ to have a preferred stimulus value $s^i_{pref}$ and tuning curve $f_i(s) = r_{max}e^{-(s-s^i_{pref})^2/(2\sigma_f)^2}$, which describes the mean firing rate response to stimulus *s*. We assume the population has $\{s^i_{pref}\}_{i=1,...,M}$ uniformly distributed in the stimulus space.

The total spike count $n_i$ for a stimulus presented over a time interval of length $T$ will have mean $\left<n_i\right>=\left<r_i\right>T=f_i(s)T$ and we assume the variability of $n_i$ around its mean in response to $s$ to be described as a Poisson process, so $ p(n_i|s) = \frac {(f_i(s)T)^{n_i}e^{-f_i(s)T}}{n_i!} $.

 Set $r_{max} = 20 Hz$, $\sigma_f=20$ and $T=1 s$.

- Generate the response $\mathbf{n}=(n_1,n_2,...,n_M)$ of a population of $M=92$ neurons (use `np.random.poisson`)
- Plot the tuning curve of the neuron with $s_{pref}=0^\circ$
- Plot the population response to stimulus $s=45^\circ$ (arrange the neurons on the x-axis based on their $s_{pref}$)

In [ ]:
r_max = 20
sigma_f = 20
T = 1
M = 92

## your solution here
# s = ...


fig, ax = plt.subplots(1, 2, figsize = (10,5))

# ax[0].plot(...)
ax[0].set_xlabel('stimulus ($^\circ$)'), ax[0].set_ylabel('f (spikes/s)')
ax[0].set_title('Tuning curve for $s_{pref}=0^\circ$')

# ax[1].scatter(...)
ax[1].set_xlabel('neurons ($s_{pref})$'), ax[1].set_ylabel('activity (spikes)')
ax[1].legend(loc='upper left', fontsize = 10)
ax[1].set_title(f'Response to $s={s}^\circ$')

fig.tight_layout()
plt.show()

# Decoders

Now we implement and compare three different decoding strategies discussed in the lectures.

## Winner-take-all

The simplest estimator of the stimulus from the population activity is the *winner-take-all* decoder, which takes as estimate $\hat{s}$ the preferred stimulus value of the neuron with the highest response: $$\hat{s} = s_j : j=\underset{i}{\mathrm{argmax}}\ n_i$$

* What are the limitations and possible failures of this strategy?

## Population vector

Another decoder is obtained by computing a weighted average of the preferred stimulus values of all neurons, with weights proportional to the responses of the respective neurons.

For a circular stimulus space (and tuning curves), the appropriate average is the [circular mean](https://en.wikipedia.org/wiki/Circular_mean). In particular, the population vector decoder computes a weighted circular average by defining the *population vector* $\mathbf{z} = \sum_i n_i\left[\cos s^i_{pref}, \sin s^i_{pref} \right]$. This corresponds to representing each neural activity with a 2D vector whose angle matches the preferred stimulus of the neuron and whose magnitude is proportional to the strenght of the response. The vector sum of the population, which is equivalent to the weighted average over the angles, will generate a vector $\mathbf{z}=[z1,z2]=[z\cos\hat{s}, z\sin\hat{s}]$ whose angle will represent the estimate of the decoded stimulus, so $$\hat{s} = \arctan\left(\frac{z2}{z1}\right)$$

* How do you think this compares to the winner-take-all in terms of performance and why? What are the limitations of this decoder?

**Note**: The population vector decoder can only be defined properly on a set of *circular* tuning curves, which is not the case we are considering here. Examples of circular tuning curves would be, for example, rectified cosine functions $f_i(s) = \frac{r_{max}}{1-r_0} [\cos(s-s_{pref}^i) - r_0]_+$ or Von Mises curves $f_i(s) = r_{max} e^{\cos(s-s_{pref}^i)/(2\sigma^2)-1}$. However, for *narrow tuning curves* and stimulus values central in the interval (not close to the boundaries), the population decoder method can be applied to Gaussian tuning curves too (indeed, von Mises tuning curves become approximately Gaussian as $\sigma\rightarrow 0$).

## Maximum Likelihood

The previous two decoders are simple estimators that might perform well but they do not take into account the variability of neurons and how this might affect the estimate (e.g. the neuron with highest average firing rate for a given stimulus will not necessarily have the highest response on a single trial). A method that explicitly takes both the form of neuronal variability and tuning curves into account is the maximum‐likelihood decoder, which consists in computing the likelihood $p(\mathbf{n}|s)$ of observing the response $\mathbf{n}$ to stimulus $s$. The estimate of the stimulus is then the value that maximises this probability:
$$ \hat{s} = \underset{s}{\mathrm{argmax}}\ p(\mathbf{n}|s) $$

Note that, since we assumed neural responses are independent, the likelihood of the population is given by $p(\mathbf{n}|s) = \prod_i p(n_i|s)$.

# Exercise 1.2 - Estimate $\hat{s}$

* Given the response $\mathbf{n}$ generated in the previous exercise, estimate $\hat{s}$ with the three different decoders. How do they compare? Do they match your expectation? Try computing the ML estimate both numerically and analytically using the result derived in lectures ($\hat{s}_{ML} = \sum_i n_i s_{pref}^i/\sum_i n_i$).

In [ ]:
## estimate s_hat with the three decoding schemes

## WTA
# s_wta = ...

## Population vector - convert from radians to degrees
# ...
# s_pop = ...

## Maximum likelihood - compute over the range of stimuli
# ...
# s_ml = ...
# s_ml_analytical = ...

print("Stimulus estimates (in degrees)")
print(f"WTA decoder: {s_wta:.2f}")
print(f"Population vector decoder: {s_pop:.2f}")
print(f"ML decoder numerical: {s_ml:.2f}")
print(f"ML decoder analytical: {s_ml_analytical:.2f}")


# Exercise 2.1 - Bias and variance of decoders

We can now compare the bias and variance of these three decoders. In order to do that, we simulate $N_{trials} = 10000$ for a fixed value of $s$ and compute (numerically) the bias $b(s) = \langle \hat{s} - s \rangle$, variance $var(s) = \langle (\hat{s} - \langle \hat{s}\rangle )^2 \rangle$, and mean squared error  $MSE(s) = \langle (\hat{s} -  s )^2 \rangle$ for each decoder (averages are with respect to $p(n|s)$ for fixed $s$). Compute these quantities for the three decoders for $s=45^\circ$.

* How do they compare? Does this match your expectation?

In [ ]:
Ntrials = 10000
r_max = 20
sigma_f = 20
T = 1
M = 92

s = 45

s_dec_wta = np.zeros([Ntrials,1])
s_dec_pop = np.zeros([Ntrials,1])
s_dec_ml = np.zeros([Ntrials,1])
s_dec_ml_analytical = np.zeros([Ntrials,1])

# ...

for t in range(Ntrials):
    # ...
    # s_dec_wta[t] = ...
    # s_dec_pop[t] = ...
    # s_dec_ml[t] = ...
    # s_dec_ml_analytical[t] = ...

# b_wta = ...
# b_pop = ...
# b_ml = ...
# b_ml_analytical = ...
# var_wta = ...
# var_pop = ...
# var_ml = ...
# var_ml_analytical = ...
# mse_wta = ...
# mse_pop = ...
# mse_ml = ...
# mse_ml_analytical = ...

print(f"Bias: \t {b_wta:.3f} (WTA), \t {b_pop:.3f} (PopVector), \t {b_ml:.3f} (ML), \t {b_ml_analytical:.3f} (ML analytical)")
print(f"Var: \t {var_wta:.3f} (WTA), \t {var_pop:.3f} (PopVector), \t {var_ml:.3f} (ML), \t {var_ml_analytical:.3f} (ML analytical)")
print(f"MSE: \t {mse_wta:.3f} (WTA), \t {mse_pop:.3f} (PopVector), \t {mse_ml:.3f} (ML), \t {mse_ml_analytical:.3f} (ML analytical)")


# Exercise 2.2 - Cramer-Rao bound

## Variance $-$ MSE $-$ Fisher Information
Suppose we are working with an **unbiased estimator**. Namely, $b(s)
= \langle \hat s \rangle - s = 0$, that is the expected value of the estimator is the true stimulus. This directly implies that the **variance** of the etimator:
$$
\text{var} (s) = \langle (\hat s - \langle \hat s\rangle)^2 \rangle = \langle (\hat s - s)^2 \rangle = MSE(s)
$$
that is the **mean squared error** of the estimator. The Cramer-Rao bound states that this value is lower-bounded by the Fisher information of the stimulus.
$$
\text{var}(s) \geq \frac{1}{I_F(s)}
$$
This becomes an equality if the estimator is [optimal](https://en.wikipedia.org/wiki/Efficiency_(statistics)). Namely, *the Fisher information is the reciprocal of the mean squared error (or variance) of an optimal estimator*.

## Fisher information of the tuning curve + Poisson noise model
As a first exercise, you will compare the Cramer-Rao lower bound to the variance of your three above estimators. For this, you will need to compute the Fisher information of the stimulus. As a reminder if $p(n|s):=\text{Poisson}(n | f(s)T) = (f(s)T)^n e^{-f(s)T}/n!$ then:
$$
I_F(s) :=  - \left\langle \frac{d^2}{ds^2} \log p(n | s) \right\rangle =  \frac{f'(s)^2}{f(s)}T
$$

Here, for the sake of analytical simplicity we consider *Gaussian tuning curves*. That is $f_i(s) = r_{max}e^{-(s-s_i)^2/(2\sigma)^2}$.

* Derive the Fisher information $I_F(s,i)$ of a single neuron tuned according to $f$.

* Additionally, for an unbiased estimator, $var(s)\geq \sum_i^M I_F(s,i):=I_F(s)$. Derive the expression for $I_F(s)$, the bound on the variance of an unbiased estimator from the population activity.

* Numerically compare the Cramer-Rao bound with the estimators' variance. What is missing from this analysis?

* Vary the number of neurons and the width of the tuning curves (separately) and plot how the Cramer-Rao bound changes.

In [ ]:
# Your solution

r_max = 20
sigma_f = 20
T = 1
M = 92

s = 45
s_pref = np.linspace(-180, 180, M)
s_range = np.linspace(-180, 180, 500)
rng = np.random.RandomState(seed=42)

# def cramer_rao(s): ...

Ms = np.arange(10, 101, 1)
bounds_M = []

for M_temp in Ms:
  s_pref_temp = np.linspace(-180, 180, M_temp)
  # def I_F(s): ...
  bounds_M.append(cramer_rao(s))

sigma_fs = np.arange(10, 21, 1)
s_pref = np.linspace(-180, 180, M)
bounds_sigma = []
for sigma_f_temp in sigma_fs:
  # def I_F(s): ...
  bounds_sigma.append(cramer_rao(s))

fig, ax = plt.subplots(1, 2, figsize = (10,5))
ax[0].plot(Ms, bounds_M)
ax[0].set_xlabel('M')
ax[0].set_ylabel('Cramer-Rao')
ax[0].set_title('$\sigma=20$')

ax[1].plot(sigma_fs, bounds_sigma)
ax[1].set_xlabel('$\sigma$')
ax[1].set_ylabel('Cramer-Rao')
ax[1].set_title('$M=92$')

plt.show()


## Biased estimators

The Cramer-Rao bound can be generalized to biased estimators:
$$
\text{var}(s) \geq \left (1+b'(s)\right )^2I_F(s)^{-1}
$$
where $\langle \hat s - s\rangle = b(s)$ is the bias.

---
**Optional** *: proof of Cramer-Rao for biased estimators.* Suppose $\langle \hat s - s\rangle = b(s)$ then $\langle \hat s - (s-b(s))\rangle = 0$ and therefore $\hat s$ is an unbiased estimate of $\bar s = s+b(s)$. We can therefore apply our previous bound to find:
$$
\text{var}(s) \geq \frac{1}{I_F(\bar s)}
$$
We would however like to obtain an expression bounding the variance in terms of the Fisher information of $s$ rather than $\bar s$. For this, we return to the definition of the Fisher information:
$$
\begin{align*}
I_F(\bar s) =& -E\left[\frac{\partial^2 }{\partial \bar s^2}\log p(x|s) \right] \\
=& -E\left[\frac{\partial}{\partial \bar s}\left (\frac{\partial \log p(x|s)}{\partial s}\frac{\partial s}{\partial \bar s}\right ) \right] \\
=& -E\left[\frac{\partial^2 \log p(x|s)}{\partial s^2}\left(\frac{\partial s}{\partial \bar s}\right)^2 + \frac{\partial \log p(x|s)}{\partial s}\frac{\partial^2 s}{\partial \bar s^2} \right] \\
= & -\left(\frac{\partial s}{\partial \bar s}\right)^2 E\left[\frac{\partial^2 \log p(x|s)}{\partial s^2}\right] - \frac{\partial^2 s}{\partial \bar s^2}E\left[\frac{\frac{\partial}{\partial s}p(x|s)}{p(x|s)} \right]
\end{align*}
$$
Now notice that $E\left[\frac{\frac{\partial}{\partial s}p(x|s)}{p(x|s)} \right]=\int_\Omega \frac{\frac{\partial}{\partial s}p(x|s)}{p(x|s)} p(x|s)dx=\frac{\partial}{\partial s}\int_\Omega p(x|s)dx=\frac{\partial}{\partial s} 1 = 0$. Furthermore $\frac{ds}{d\bar s}=(1+b'(s))^{-1}$ and $E\left[\frac{\partial^2 \log p(x|s)}{\partial s^2}\right]$ is simply the Fisher information of the stimulus $I_F(s)$. Thus:
$$
I_F(\bar s) = (1+b'(s))^{-2} I_F(s)
$$
and finally, using the expression we started with:
$$
\text{var}(s) \geq \left (1+b'(s)\right )^2I_F(s)^{-1}
$$

---

* Derive an expression for the variance in terms of the MSE and bias.

* Use the above expression to bound the MSE.

* In lectures we computed the Fisher information for the limit of continuously many neurons with density $\rho = M/360^\circ$ and found that $I_F(s) \rightarrow \frac{\sqrt{2\pi} \rho r_{max} T}{\sigma}$. Compare this quantity numerically to the variance obtained above for a finite number of neurons $M$. Try varying $M$ to check the validity of the approximation.